In [1]:
import qpms
import numpy as np
from numpy import newaxis as nx
import math
import cmath
from scipy.constants import c, e as eV, hbar
s2 = math.sqrt(2)
J_scat = 3

In [2]:
epsilon_b = 2.3104
x_period = 560e-9
y_period = 560e-9
cdn = c/ math.sqrt(epsilon_b)
maxlayer=4

omegalist=np.linspace(0,5e15,200)
#omegalist=np.linspace(0,3,3.2)*eV/hbar
#omegalist = np.array([1.5,])*eV/hbar

# reciprocal lattice basis
B1 = np.array((2*np.pi/x_period,0))
B2 = np.array((0,2*np.pi/y_period))

bz_0 = np.array((0,0,))
bz_X1 = B1/2
bz_X2 = B2/2
bz_M = (B1+B2)/2

In [3]:
kdensity = 100
k0Mlist = bz_0 + (bz_M-bz_0) * np.linspace(0,1,kdensity)[:,nx]
kMX1list = bz_M + (bz_X1-bz_M) * np.linspace(0,1,kdensity)[:,nx]
kX10list = bz_X1 + (bz_0-bz_X1) * np.linspace(0,1,kdensity)[:,nx]
k0X2list = bz_0 + (bz_X2-bz_0) * np.linspace(0,1,kdensity)[:,nx]
kX2Mlist = bz_X2 + (bz_M-bz_X2) * np.linspace(0,1,kdensity)[:,nx]
klist = np.concatenate((k0Mlist,kMX1list,kX10list,k0X2list,kX2Mlist), axis=0)
kxmaplist = np.concatenate((np.array([0]),np.cumsum(np.linalg.norm(np.diff(klist, axis=0), axis=-1))))

In [4]:
#centers=np.array([bz_0,B1,B2,-B1,-B2,B2-B1,-B2+B1,-B2-B1,B2+B1,-2*B1,-2*B2,-2*B2-B1,-2*B1-B2,-2*B1-2*B2])
#centers2=qpms.generate_trianglepoints(maxlayer, v3d = False, include_origin= True)*4*np.pi/3/hexside
#rot90 = np.array([[0,-1],[1,0]])
#centers2=np.dot(centers2,rot90)
centers2 = qpms.generateLattice(B1,B2, maxlayer=maxlayer, include_origin=True)
corners = qpms.cellCornersWS(B1,B2)

In [ ]:
print(corners)
print(centers2)

[[ 5609986.88141034  5609986.88141034]
 [-5609986.88141034  5609986.88141034]
 [-5609986.88141034 -5609986.88141034]
 [ 5609986.88141034 -5609986.88141034]]
[[        0.                 0.        ]
 [ 11219973.76282069         0.        ]
 [ 11219973.76282069  11219973.76282069]
 [ 22439947.52564138         0.        ]
 [ 11219973.76282069  22439947.52564138]
 [ 22439947.52564138  11219973.76282069]
 [ 33659921.28846207         0.        ]
 [ 11219973.76282069  33659921.28846207]
 [ 22439947.52564138  22439947.52564138]
 [ 33659921.28846207  11219973.76282069]
 [ 44879895.05128276         0.        ]
 [        0.          11219973.76282069]
 [-11219973.76282069  11219973.76282069]
 [        0.          22439947.52564138]
 [-22439947.52564138  11219973.76282069]
 [-11219973.76282069  22439947.52564138]
 [        0.          33659921.28846207]
 [-33659921.28846207  11219973.76282069]
 [-22439947.52564138  22439947.52564138]
 [-11219973.76282069  33659921.28846207]
 [        0.          4

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.path import Path
import matplotlib.patches as patches
cmap = matplotlib.cm.prism
colormax = np.amax(np.linalg.norm(centers2,axis=0))
for oi in range(omegalist.shape[0]):
    omega = omegalist[oi]
    f, axes = plt.subplots(1,2, figsize=(10,4.52))
    ax = axes[0]
    #sc = ax.scatter(k2list[:,0], k2list[:,1], c = np.clip(np.abs(minsvTElistarr[oi,...,minN]),0,1), lw=0)
    for center in centers2:
        circle=plt.Circle((center[0],center[1]),omega/cdn, facecolor='none', edgecolor=cmap(np.linalg.norm(center)/colormax),lw=1)
        ax.add_artist(circle)
    verts = [corners[i % len(corners)]  for i in range(len(corners) +1)]
    codes = [Path.MOVETO] + [Path.LINETO]*(len(corners)-1) + [Path.CLOSEPOLY]
    path = Path(verts, codes)
    patch = patches.PathPatch(path, facecolor='none', edgecolor='black',  lw=1)
    ax.add_patch(patch)
    ax.set_xlim([-10e6,10e6])
    ax.set_ylim([-10e6,10e6])
    ax.set_xticks([]) 
    ax.set_yticks([]) 
    #ax.scatter(centers2[:,0], centers2[:,1])
    print(omega/cdn)
    #f.colorbar(sc,ax=ax)
    
    ax = axes[1]
    for center in centers2:
        ax.plot(kxmaplist, np.linalg.norm(klist-center,axis=-1)*cdn, '-', color=cmap(np.linalg.norm(center)/colormax))

    #ax.set_xlim([np.min(kxmlarr),np.max(kxmlarr)])
    #ax.set_ylim([np.min(omegalist),np.max(omegalist)])
    xticklist = [0, kxmaplist[len(k0Mlist)-1], kxmaplist[len(k0Mlist)+len(kMX1list)-1], kxmaplist[len(k0Mlist)+len(kMX1list)+len(kX10list)-1], kxmaplist[len(k0Mlist)+len(kMX1list)+len(kX10list)+len(k0X2list)-1], kxmaplist[len(k0Mlist)+len(kMX1list)+len(kX10list)+len(k0X2list)+len(kX2Mlist)-1]]
    ax.set_xticks(xticklist)
    for xt in xticklist:
        ax.axvline(xt, ls='dotted', lw=0.3,c='k')
    ax.set_xticklabels(['Γ', 'M', 'X', 'Γ', 'X\'','M'])
    ax.axhline(omega, c='black')
    ax.set_ylim([0,5e15])
    ax2 = ax.twinx()
    ax2.set_ylim([ax.get_ylim()[0]/eV*hbar,ax.get_ylim()[1]/eV*hbar])
    f.savefig("diforder-sq-%.4f.pdf"%(omega/eV*hbar,))


0.0
127391.31273899274
254782.62547798548
382173.9382169783
509565.25095597096
636956.5636949637
764347.8764339566
891739.1891729492
1019130.5019119419
1146521.814650935
1273913.1273899274


In [ ]:
import itertools
list(itertools.chain([1, 3, i] for i in range(4)))